### Importing Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,Bidirectional
from keras.models import Model
from keras.models import Sequential

# task_1

### Handling Pre-processed data

In [2]:
data = pd.read_csv('../preprocess_data.csv')
data.drop(['task_2','text'], axis=1, inplace=True)
data.head()

,_id,task_1,task_3,task_4,text_clean
0,Gujarati_image_1618.jpg,Sarcastic,Vulgar,Abusive,છોકર). ટીચર તમાર તાજમહેલ\r\n\r\nદેખ/ય છે.\r\n\...
1,Gujarati_image_31.jpg,Sarcastic,Vulgar,Abusive,છોકરો : ના.\r\n છોકરી : કેમ?\r\n \r\n છોકરી : ...
2,Gujarati_image_1144.jpg,Sarcastic,Vulgar,Abusive,"છોકરીઓ ગમે તેટલી\r\n ચાલક હોય,\r\n \r\n પણ છોક..."
3,Gujarati_image_1184.jpg,Sarcastic,Vulgar,Abusive,"દોસ્તી કરો,પ્રેમ કરો, વફા કરો...\r\n અને બહુ મ..."
4,Gujarati_image_1643.jpg,Sarcastic,Vulgar,Abusive,"છોકરીઓ ગમે તેટલી\r\nચાલક હોય,\r\n\r\nપણ છોકરા ..."


In [3]:
test_data = pd.read_csv('../preprocess_test_data.csv') 
test_data.drop(['task_2','text'], axis=1, inplace=True)
test_data = test_data.drop(['Unnamed: 0'],axis=1)
test_data.head()

,_id,task_1,task_3,task_4,text_clean
0,Gujarati_image_1225.jpg,Non-Sarcastic,Non Vulgar,Non-abusive,॥વિંદેશીગામડિયો\r\n અ |
1,Gujarati_image_1583.jpg,Sarcastic,Vulgar,Abusive,ટીચર : સૌથી વધારે દુખાવો ક્યારે\r\nથાય?\r\nછોક...
2,Gujarati_image_1502.jpg,Sarcastic,Vulgar,Abusive,પતિ: તુંમને જરાય પ્રેમ\r\nનથી કરતી...\r\n\r\nપ...
3,Gujarati_image_1487.jpg,Sarcastic,Vulgar,Abusive,આખા ગોમ ના લોડા\r\nભોસ મા ભરી ને બેઠી\r\nહોય અ...
4,Gujarati_image_1497.jpg,Non-Sarcastic,Vulgar,Abusive,મિનરલ વોટર સિવાય ક્યારેય\r\nબીજું\r\nપાણી નો પ...


In [4]:
sentences = data['text_clean'].astype(str)
tokenizer = Tokenizer(num_words = 1500,split=' ')
tokenizer.fit_on_texts(sentences)
sequence = tokenizer.texts_to_sequences(sentences)

In [5]:
test_sentences = test_data['text_clean'].astype(str)
test_sequence = tokenizer.texts_to_sequences(test_sentences)

In [6]:
max_seq_len = 2500

index_of_words = tokenizer.word_index
print("No of unique words : ",len(index_of_words))

X = pad_sequences(sequence , maxlen = max_seq_len )
Y = data['task_1']

print(X)

No of unique words :  7000
[[   0    0    0 ...  536  134    1]
 [   0    0    0 ...  246 1062  318]
 [   0    0    0 ... 1067 1068  174]
 ...
 [   0    0    0 ...    0    0    5]
 [   0    0    0 ...    0    0    5]
 [   0    0    0 ...    1   17  433]]


In [7]:
test_X = pad_sequences(test_sequence , maxlen = max_seq_len )
test_Y = test_data['task_1']

print(test_X)

[[  0   0   0 ...   0   0 648]
 [  0   0   0 ... 206 394   1]
 [  0   0   0 ...   2   1   1]
 ...
 [  0   0   0 ...   1   1 324]
 [  0   0   0 ...  40   2   1]
 [  0   0   0 ...   0   0   5]]


In [8]:
embed_dim = 256
vocabSize = len(index_of_words)
lstm_out = 64

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15, random_state = 0)
Y_true = Y_test
Y_train = pd.get_dummies(Y_train).values
Y_test = pd.get_dummies(Y_test).values

In [10]:
test_Y_true = test_Y
test_Y = pd.get_dummies(test_Y).values
print("test_Y:", test_Y)

test_Y: [[ True False]
 [False  True]
 [False  True]
 ...
 [ True False]
 [ True False]
 [ True False]]


# MODEL 1

In [11]:
model = Sequential()
model.add(Embedding(vocabSize, embed_dim,input_length = 2500))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2500, 256)         1792000   
                                                                 
 lstm (LSTM)                 (None, 64)                82176     
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 1874306 (7.15 MB)
Trainable params: 1874306 (7.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [12]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    "hasoc_a1.h5",
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False, 
    mode='auto')

In [13]:
model.fit(X_train, Y_train, batch_size = 32, epochs = 10, validation_data=(X_test,Y_test), callbacks=[checkpoint])

Epoch 1/10
24/24 [==============================] - ETA: 0s - loss: 0.6109 - accuracy: 0.7351
Epoch 1: val_loss improved from inf to 0.52641, saving model to hasoc_a1.h5
24/24 [==============================] - 126s 5s/step - loss: 0.6109 - accuracy: 0.7351 - val_loss: 0.5264 - val_accuracy: 0.7836
Epoch 2/10


C:\Users\8888\Anaconda3\envs\pythonProject11\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


24/24 [==============================] - ETA: 0s - loss: 0.5496 - accuracy: 0.7483
Epoch 2: val_loss improved from 0.52641 to 0.50465, saving model to hasoc_a1.h5
24/24 [==============================] - 124s 5s/step - loss: 0.5496 - accuracy: 0.7483 - val_loss: 0.5046 - val_accuracy: 0.7836
Epoch 3/10
24/24 [==============================] - ETA: 0s - loss: 0.5002 - accuracy: 0.7576
Epoch 3: val_loss improved from 0.50465 to 0.49123, saving model to hasoc_a1.h5
24/24 [==============================] - 125s 5s/step - loss: 0.5002 - accuracy: 0.7576 - val_loss: 0.4912 - val_accuracy: 0.7910
Epoch 4/10
24/24 [==============================] - ETA: 0s - loss: 0.3910 - accuracy: 0.8159
Epoch 4: val_loss did not improve from 0.49123
24/24 [==============================] - 124s 5s/step - loss: 0.3910 - accuracy: 0.8159 - val_loss: 0.5532 - val_accuracy: 0.7687
Epoch 5/10
24/24 [==============================] - ETA: 0s - loss: 0.3012 - accuracy: 0.8967
Epoch 5: val_loss did not improve from

In [14]:
model.load_weights('hasoc_a1.h5')
model.evaluate(X_test,Y_test)

5/5 [==============================] - 3s 525ms/step - loss: 0.4912 - accuracy: 0.7910


[0.49123457074165344, 0.7910447716712952]

In [15]:
Y_pred = model.predict(test_X)

19/19 [==============================] - 12s 610ms/step


In [16]:
y_actual = []
for i in test_Y_true:
    if i == 'Sarcastic':
        y_actual.append(1)
    else :
        y_actual.append(0)

pred_class = []
for i in Y_pred:
    pred_class.append(np.argmax(i))

In [17]:
print(classification_report(y_actual, pred_class))

              precision    recall  f1-score   support

           0       0.33      0.00      0.01       410
           1       0.32      0.98      0.48       194

    accuracy                           0.32       604
   macro avg       0.33      0.49      0.24       604
weighted avg       0.33      0.32      0.16       604



In [18]:
print("Y_pred:", Y_pred)

Y_pred: [[0.10724969 0.8927503 ]
 [0.35344046 0.64655954]
 [0.03545646 0.9645435 ]
 ...
 [0.08703656 0.9129634 ]
 [0.09556682 0.9044332 ]
 [0.13938175 0.86061823]]


In [19]:
print("pred_class:", pred_class)

pred_class: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [20]:
pred_actual = []
for i in pred_class:
    if i == 1:
        pred_actual.append('Sarcastic')
    else :
        pred_actual.append('Non-Sarcastic')

In [21]:
test_data = test_data[["_id"]]
test_data["label"] = pred_actual
test_data.to_csv('dl_lstm_a.csv',index=False)
test_data.head()

,_id,label
0,Gujarati_image_1225.jpg,Sarcastic
1,Gujarati_image_1583.jpg,Sarcastic
2,Gujarati_image_1502.jpg,Sarcastic
3,Gujarati_image_1487.jpg,Sarcastic
4,Gujarati_image_1497.jpg,Sarcastic


# MODEL 2

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential()

# Embedding层，增加防止过拟合的Dropout
model.add(Embedding(input_dim=vocabSize, output_dim=embed_dim, input_length=2500))
model.add(Dropout(0.3))

# LSTM层，增加recurrent_dropout 和 output dropout
model.add(LSTM(units=lstm_out, dropout=0.3, recurrent_dropout=0.3, return_sequences=False))

# Batch Normalization增强泛化
model.add(BatchNormalization())

# 全连接层，Softmax输出2分类，建议用categorical_crossentropy
model.add(Dense(2, activation='softmax'))

# 编译
optimizer = Adam(learning_rate=0.001)  # 学习率也可调整
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2500, 256)         1792000   
                                                                 
 dropout (Dropout)           (None, 2500, 256)         0         
                                                                 
 lstm (LSTM)                 (None, 64)                82176     
                                                                 
 batch_normalization (Batch  (None, 64)                256       
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 1874562 (7.15 MB)
Trainable params: 1874434 (7.15 MB)
Non-trainable params: 128 (512.00 Byte)
______________

In [12]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("hasoc_b1.h5", monitor='val_loss', verbose=1, save_best_only=True,
save_weights_only=False, mode='auto')

In [13]:
model.fit(X_train, Y_train, batch_size = 32, epochs = 10, validation_data = (X_test,Y_test), callbacks = [checkpoint])

Epoch 1/10
24/24 [==============================] - ETA: 0s - loss: 0.7247 - accuracy: 0.5113
Epoch 1: val_loss improved from inf to 0.60993, saving model to hasoc_b1.h5
24/24 [==============================] - 125s 5s/step - loss: 0.7247 - accuracy: 0.5113 - val_loss: 0.6099 - val_accuracy: 0.7836
Epoch 2/10


C:\Users\8888\Anaconda3\envs\pythonProject11\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


24/24 [==============================] - ETA: 0s - loss: 0.5304 - accuracy: 0.7642
Epoch 2: val_loss improved from 0.60993 to 0.56328, saving model to hasoc_b1.h5
24/24 [==============================] - 120s 5s/step - loss: 0.5304 - accuracy: 0.7642 - val_loss: 0.5633 - val_accuracy: 0.7836
Epoch 3/10
24/24 [==============================] - ETA: 0s - loss: 0.4138 - accuracy: 0.8371
Epoch 3: val_loss improved from 0.56328 to 0.53399, saving model to hasoc_b1.h5
24/24 [==============================] - 122s 5s/step - loss: 0.4138 - accuracy: 0.8371 - val_loss: 0.5340 - val_accuracy: 0.7836
Epoch 4/10
24/24 [==============================] - ETA: 0s - loss: 0.3294 - accuracy: 0.8583
Epoch 4: val_loss improved from 0.53399 to 0.51653, saving model to hasoc_b1.h5
24/24 [==============================] - 123s 5s/step - loss: 0.3294 - accuracy: 0.8583 - val_loss: 0.5165 - val_accuracy: 0.7836
Epoch 5/10
24/24 [==============================] - ETA: 0s - loss: 0.2735 - accuracy: 0.8795
Epoch

In [14]:
model.load_weights('hasoc_b1.h5')
model.evaluate(X_test,Y_test)

5/5 [==============================] - 3s 572ms/step - loss: 0.5134 - accuracy: 0.7836


[0.513394296169281, 0.7835820913314819]

In [15]:
Y_pred = model.predict(test_X)

19/19 [==============================] - 11s 596ms/step


In [16]:
y_actual = []
for i in test_Y_true:
    if i == 'Sarcastic':
        y_actual.append(1)
    else :
        y_actual.append(0)

pred_class = []
for i in Y_pred:
    pred_class.append(np.argmax(i))

In [17]:
print(classification_report(y_actual , pred_class))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       410
           1       0.32      1.00      0.49       194

    accuracy                           0.32       604
   macro avg       0.16      0.50      0.24       604
weighted avg       0.10      0.32      0.16       604



C:\Users\8888\Anaconda3\envs\pythonProject11\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\8888\Anaconda3\envs\pythonProject11\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\8888\Anaconda3\envs\pythonProject11\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [18]:
pred_actual = []
for i in pred_class:
    if i == 1:
        pred_actual.append('Sarcastic')
    else :
        pred_actual.append('Non-Sarcastic')

In [19]:
test_data = test_data[["_id"]]
test_data["label"] = pred_actual
test_data.to_csv('dl_lstm_b.csv',index=False)
test_data.head()

,_id,label
0,Gujarati_image_1225.jpg,Sarcastic
1,Gujarati_image_1583.jpg,Sarcastic
2,Gujarati_image_1502.jpg,Sarcastic
3,Gujarati_image_1487.jpg,Sarcastic
4,Gujarati_image_1497.jpg,Sarcastic


# MODEL 3

In [11]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    "hasoc_c1.h5",
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False, 
    mode='auto')

In [12]:
model = Sequential()
model.add(Embedding(vocabSize, embed_dim, input_length=2500))
model.add(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

from sklearn.utils.class_weight import compute_class_weight
# 假设Y_train已独热编码
class_weights = compute_class_weight(class_weight='balanced', classes=np.array([0, 1]), y=np.argmax(Y_train, axis=1))
class_weight_dict = dict(enumerate(class_weights))

model.fit(X_train, Y_train, epochs=10, batch_size=64, class_weight=class_weight_dict, validation_data=(X_test,Y_test), callbacks=[checkpoint])

Epoch 1/10
12/12 [==============================] - ETA: 0s - loss: 0.6941 - accuracy: 0.4768 
Epoch 1: val_loss improved from inf to 0.67589, saving model to hasoc_c1.h5
12/12 [==============================] - 193s 16s/step - loss: 0.6941 - accuracy: 0.4768 - val_loss: 0.6759 - val_accuracy: 0.7015
Epoch 2/10


C:\Users\8888\Anaconda3\envs\pythonProject11\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


12/12 [==============================] - ETA: 0s - loss: 0.6738 - accuracy: 0.7497 
Epoch 2: val_loss improved from 0.67589 to 0.65912, saving model to hasoc_c1.h5
12/12 [==============================] - 188s 16s/step - loss: 0.6738 - accuracy: 0.7497 - val_loss: 0.6591 - val_accuracy: 0.7164
Epoch 3/10
12/12 [==============================] - ETA: 0s - loss: 0.6313 - accuracy: 0.7020 
Epoch 3: val_loss improved from 0.65912 to 0.56992, saving model to hasoc_c1.h5
12/12 [==============================] - 201s 17s/step - loss: 0.6313 - accuracy: 0.7020 - val_loss: 0.5699 - val_accuracy: 0.8060
Epoch 4/10
12/12 [==============================] - ETA: 0s - loss: 0.5243 - accuracy: 0.8000 
Epoch 4: val_loss did not improve from 0.56992
12/12 [==============================] - 199s 17s/step - loss: 0.5243 - accuracy: 0.8000 - val_loss: 0.6379 - val_accuracy: 0.5970
Epoch 5/10
12/12 [==============================] - ETA: 0s - loss: 0.3710 - accuracy: 0.8159 
Epoch 5: val_loss did not impro

In [13]:
model.load_weights('hasoc_c1.h5')
model.evaluate(X_test,Y_test)

5/5 [==============================] - 3s 649ms/step - loss: 0.5699 - accuracy: 0.8060


[0.5699222087860107, 0.8059701323509216]

In [14]:
Y_pred = model.predict(test_X)

19/19 [==============================] - 14s 739ms/step


In [15]:
y_actual = []
for i in test_Y_true:
    if i == 'Sarcastic':
        y_actual.append(1)
    else :
        y_actual.append(0)

pred_class = []
for i in Y_pred:
    pred_class.append(np.argmax(i))

In [16]:
print(classification_report(y_actual , pred_class))

              precision    recall  f1-score   support

           0       0.66      0.05      0.10       410
           1       0.32      0.94      0.48       194

    accuracy                           0.34       604
   macro avg       0.49      0.50      0.29       604
weighted avg       0.55      0.34      0.22       604



In [17]:
print("Y_pred:",Y_pred)

Y_pred: [[0.4008175  0.5991824 ]
 [0.5229386  0.47706142]
 [0.32581317 0.6741869 ]
 ...
 [0.36569163 0.6343084 ]
 [0.4074059  0.5925941 ]
 [0.44256213 0.55743796]]


In [18]:
pred_actual = []
for i in pred_class:
    if i == 1:
        pred_actual.append('Sarcastic')
    else :
        pred_actual.append('Non-Sarcastic')

In [19]:
test_data = test_data[["_id"]]
test_data["label"] = pred_actual
test_data.to_csv('dl_lstm_c.csv',index=False)
test_data.head()

,_id,label
0,Gujarati_image_1225.jpg,Sarcastic
1,Gujarati_image_1583.jpg,Non-Sarcastic
2,Gujarati_image_1502.jpg,Sarcastic
3,Gujarati_image_1487.jpg,Sarcastic
4,Gujarati_image_1497.jpg,Non-Sarcastic


# task_3

# Handling Pre-processed data

In [2]:
data = pd.read_csv('../preprocess_data.csv')
data.drop(['task_2','text'], axis=1, inplace=True)
data.head()

,_id,task_1,task_3,task_4,text_clean
0,Gujarati_image_1618.jpg,Sarcastic,Vulgar,Abusive,છોકર). ટીચર તમાર તાજમહેલ\r\n\r\nદેખ/ય છે.\r\n\...
1,Gujarati_image_31.jpg,Sarcastic,Vulgar,Abusive,છોકરો : ના.\r\n છોકરી : કેમ?\r\n \r\n છોકરી : ...
2,Gujarati_image_1144.jpg,Sarcastic,Vulgar,Abusive,"છોકરીઓ ગમે તેટલી\r\n ચાલક હોય,\r\n \r\n પણ છોક..."
3,Gujarati_image_1184.jpg,Sarcastic,Vulgar,Abusive,"દોસ્તી કરો,પ્રેમ કરો, વફા કરો...\r\n અને બહુ મ..."
4,Gujarati_image_1643.jpg,Sarcastic,Vulgar,Abusive,"છોકરીઓ ગમે તેટલી\r\nચાલક હોય,\r\n\r\nપણ છોકરા ..."


In [3]:
test_data = pd.read_csv('../preprocess_test_data.csv') 
test_data.drop(['task_2','text'], axis=1, inplace=True)
test_data = test_data.drop(['Unnamed: 0'],axis=1)
test_data.head()

,_id,task_1,task_3,task_4,text_clean
0,Gujarati_image_1225.jpg,Non-Sarcastic,Non Vulgar,Non-abusive,॥વિંદેશીગામડિયો\r\n અ |
1,Gujarati_image_1583.jpg,Sarcastic,Vulgar,Abusive,ટીચર : સૌથી વધારે દુખાવો ક્યારે\r\nથાય?\r\nછોક...
2,Gujarati_image_1502.jpg,Sarcastic,Vulgar,Abusive,પતિ: તુંમને જરાય પ્રેમ\r\nનથી કરતી...\r\n\r\nપ...
3,Gujarati_image_1487.jpg,Sarcastic,Vulgar,Abusive,આખા ગોમ ના લોડા\r\nભોસ મા ભરી ને બેઠી\r\nહોય અ...
4,Gujarati_image_1497.jpg,Non-Sarcastic,Vulgar,Abusive,મિનરલ વોટર સિવાય ક્યારેય\r\nબીજું\r\nપાણી નો પ...


In [4]:
sentences = data['text_clean'].astype(str)
tokenizer = Tokenizer(num_words = 1500,split=' ')
tokenizer.fit_on_texts(sentences)
sequence = tokenizer.texts_to_sequences(sentences)

In [5]:
test_sentences = test_data['text_clean'].astype(str)
test_sequence = tokenizer.texts_to_sequences(test_sentences)

In [6]:
max_seq_len = 2500

index_of_words = tokenizer.word_index
print("No of unique words : ",len(index_of_words))

X = pad_sequences(sequence , maxlen = max_seq_len )
Y = data['task_3']

print(X)

No of unique words :  7000
[[   0    0    0 ...  536  134    1]
 [   0    0    0 ...  246 1062  318]
 [   0    0    0 ... 1067 1068  174]
 ...
 [   0    0    0 ...    0    0    5]
 [   0    0    0 ...    0    0    5]
 [   0    0    0 ...    1   17  433]]


In [7]:
test_X = pad_sequences(test_sequence , maxlen = max_seq_len )
test_Y = test_data['task_3']

print(test_X)

[[  0   0   0 ...   0   0 648]
 [  0   0   0 ... 206 394   1]
 [  0   0   0 ...   2   1   1]
 ...
 [  0   0   0 ...   1   1 324]
 [  0   0   0 ...  40   2   1]
 [  0   0   0 ...   0   0   5]]


In [8]:
embed_dim = 256
vocabSize = len(index_of_words)
lstm_out = 64

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15, random_state = 0)
Y_true = Y_test
Y_train = pd.get_dummies(Y_train).values
Y_test = pd.get_dummies(Y_test).values

In [10]:
test_Y_true = test_Y
test_Y = pd.get_dummies(test_Y).values
print("test_Y:",test_Y)

test_Y: [[ True False]
 [False  True]
 [False  True]
 ...
 [ True False]
 [ True False]
 [ True False]]


# MODEL 1

In [12]:
model = Sequential()
model.add(Embedding(vocabSize, embed_dim,input_length = 2500))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2500, 256)         1792000   
                                                                 
 lstm (LSTM)                 (None, 64)                82176     
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 1874306 (7.15 MB)
Trainable params: 1874306 (7.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [13]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    "hasoc_a3.h5",
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False, 
    mode='auto')

In [14]:
model.fit(X_train, Y_train, batch_size = 32, epochs = 10, validation_data = (X_test,Y_test), callbacks=[checkpoint])

Epoch 1/10
24/24 [==============================] - ETA: 0s - loss: 0.6070 - accuracy: 0.6596
Epoch 1: val_loss improved from inf to 0.56218, saving model to hasoc_a3.h5
24/24 [==============================] - 98s 4s/step - loss: 0.6070 - accuracy: 0.6596 - val_loss: 0.5622 - val_accuracy: 0.6567
Epoch 2/10


C:\Users\8888\Anaconda3\envs\pythonProject11\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


24/24 [==============================] - ETA: 0s - loss: 0.5493 - accuracy: 0.7589
Epoch 2: val_loss improved from 0.56218 to 0.52034, saving model to hasoc_a3.h5
24/24 [==============================] - 95s 4s/step - loss: 0.5493 - accuracy: 0.7589 - val_loss: 0.5203 - val_accuracy: 0.8060
Epoch 3/10
24/24 [==============================] - ETA: 0s - loss: 0.4582 - accuracy: 0.8358
Epoch 3: val_loss improved from 0.52034 to 0.46331, saving model to hasoc_a3.h5
24/24 [==============================] - 96s 4s/step - loss: 0.4582 - accuracy: 0.8358 - val_loss: 0.4633 - val_accuracy: 0.7985
Epoch 4/10
24/24 [==============================] - ETA: 0s - loss: 0.3104 - accuracy: 0.8861
Epoch 4: val_loss did not improve from 0.46331
24/24 [==============================] - 96s 4s/step - loss: 0.3104 - accuracy: 0.8861 - val_loss: 0.5014 - val_accuracy: 0.7687
Epoch 5/10
24/24 [==============================] - ETA: 0s - loss: 0.2103 - accuracy: 0.9325
Epoch 5: val_loss did not improve from 0.

In [15]:
model.load_weights('hasoc_a3.h5')
model.evaluate(X_test,Y_test)

5/5 [==============================] - 2s 443ms/step - loss: 0.4633 - accuracy: 0.7985


[0.4633116126060486, 0.7985074520111084]

In [16]:
Y_pred = model.predict(test_X)

19/19 [==============================] - 10s 506ms/step


In [17]:
y_actual = []
for i in test_Y_true:
    if i =='Vulgar':
        y_actual.append(1)
    else :
        y_actual.append(0)

pred_class = []
for i in Y_pred:
    pred_class.append(np.argmax(i))

In [18]:
print(classification_report(y_actual, pred_class))

              precision    recall  f1-score   support

           0       0.89      0.83      0.86       496
           1       0.41      0.54      0.46       108

    accuracy                           0.78       604
   macro avg       0.65      0.68      0.66       604
weighted avg       0.80      0.78      0.79       604



In [19]:
print("Y_pred:", Y_pred)

Y_pred: [[0.9903398  0.00966018]
 [0.04106864 0.9589314 ]
 [0.6246746  0.3753254 ]
 ...
 [0.96826684 0.0317331 ]
 [0.961996   0.03800403]
 [0.9824645  0.01753556]]


In [20]:
print("pred_class:", pred_class)

pred_class: [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 

In [21]:
pred_actual = []
for i in pred_class:
    if i == 1:
        pred_actual.append('Vulgar')
    else :
        pred_actual.append('Non Vulgar')

In [22]:
test_data = test_data[["_id"]]
test_data["label"] = pred_actual
test_data.to_csv('dl_lstm_a3.csv', index=False)
test_data.head()

,_id,label
0,Gujarati_image_1225.jpg,Non Vulgar
1,Gujarati_image_1583.jpg,Vulgar
2,Gujarati_image_1502.jpg,Non Vulgar
3,Gujarati_image_1487.jpg,Non Vulgar
4,Gujarati_image_1497.jpg,Non Vulgar


# MODEL 2

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential()

# Embedding层，增加防止过拟合的Dropout
model.add(Embedding(input_dim=vocabSize, output_dim=embed_dim, input_length=2500))
model.add(Dropout(0.3))

# LSTM层，增加recurrent_dropout 和 output dropout
model.add(LSTM(units=lstm_out, dropout=0.3, recurrent_dropout=0.3, return_sequences=False))

# Batch Normalization增强泛化
model.add(BatchNormalization())

# 全连接层，Softmax输出2分类，建议用categorical_crossentropy
model.add(Dense(2, activation='softmax'))

# 编译
optimizer = Adam(learning_rate=0.001)  # 学习率也可调整
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 2500, 256)         1792000   
                                                                 
 dropout (Dropout)           (None, 2500, 256)         0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                82176     
                                                                 
 batch_normalization (Batch  (None, 64)                256       
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 1874562 (7.15 MB)
Trainable params: 1874434 (7.15 MB)
Non-trainable params: 128 (512.00 Byte)
____________

In [24]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    "hasoc_b3.h5",
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False, 
    mode='auto')

In [25]:
model.fit(X_train, Y_train, batch_size = 32, epochs = 10, validation_data = (X_test,Y_test), callbacks = [checkpoint])

Epoch 1/10
24/24 [==============================] - ETA: 0s - loss: 0.6636 - accuracy: 0.6172
Epoch 1: val_loss improved from inf to 0.64467, saving model to hasoc_b3.h5
24/24 [==============================] - 104s 4s/step - loss: 0.6636 - accuracy: 0.6172 - val_loss: 0.6447 - val_accuracy: 0.6567
Epoch 2/10


C:\Users\8888\Anaconda3\envs\pythonProject11\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


24/24 [==============================] - ETA: 0s - loss: 0.4786 - accuracy: 0.7894
Epoch 2: val_loss improved from 0.64467 to 0.62699, saving model to hasoc_b3.h5
24/24 [==============================] - 101s 4s/step - loss: 0.4786 - accuracy: 0.7894 - val_loss: 0.6270 - val_accuracy: 0.6567
Epoch 3/10
24/24 [==============================] - ETA: 0s - loss: 0.3241 - accuracy: 0.8623
Epoch 3: val_loss improved from 0.62699 to 0.61049, saving model to hasoc_b3.h5
24/24 [==============================] - 101s 4s/step - loss: 0.3241 - accuracy: 0.8623 - val_loss: 0.6105 - val_accuracy: 0.6567
Epoch 4/10
24/24 [==============================] - ETA: 0s - loss: 0.2255 - accuracy: 0.9232
Epoch 4: val_loss improved from 0.61049 to 0.60861, saving model to hasoc_b3.h5
24/24 [==============================] - 103s 4s/step - loss: 0.2255 - accuracy: 0.9232 - val_loss: 0.6086 - val_accuracy: 0.6567
Epoch 5/10
24/24 [==============================] - ETA: 0s - loss: 0.1946 - accuracy: 0.9325
Epoch

In [26]:
model.load_weights('hasoc_b3.h5')
model.evaluate(X_test,Y_test)

5/5 [==============================] - 3s 538ms/step - loss: 0.5216 - accuracy: 0.7388


[0.52164626121521, 0.7388059496879578]

In [27]:
Y_pred = model.predict(test_X)

19/19 [==============================] - 12s 609ms/step


In [28]:
y_actual = []
for i in test_Y_true:
    if i == 'Vulgar':
        y_actual.append(1)
    else :
        y_actual.append(0)

pred_class = []
for i in Y_pred:
    pred_class.append(np.argmax(i))

In [29]:
print(classification_report(y_actual, pred_class))

              precision    recall  f1-score   support

           0       0.87      0.91      0.89       496
           1       0.48      0.36      0.41       108

    accuracy                           0.81       604
   macro avg       0.67      0.64      0.65       604
weighted avg       0.80      0.81      0.80       604



In [30]:
pred_actual = []
for i in pred_class:
    if i == 1:
        pred_actual.append('Vulgar')
    else :
        pred_actual.append('Non Vulgar')

In [31]:
test_data = test_data[["_id"]]
test_data["label"] = pred_actual
test_data.to_csv('dl_lstm_b3.csv',index=False)
test_data.head()

,_id,label
0,Gujarati_image_1225.jpg,Non Vulgar
1,Gujarati_image_1583.jpg,Vulgar
2,Gujarati_image_1502.jpg,Non Vulgar
3,Gujarati_image_1487.jpg,Non Vulgar
4,Gujarati_image_1497.jpg,Non Vulgar


# MODEL 3

In [11]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    "hasoc_c3.h5",
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False, 
    mode='auto')

In [12]:
model = Sequential()
model.add(Embedding(vocabSize, embed_dim, input_length=2500))
model.add(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

from sklearn.utils.class_weight import compute_class_weight
# 假设Y_train已独热编码
class_weights = compute_class_weight(class_weight='balanced', classes=np.array([0, 1]), y=np.argmax(Y_train, axis=1))
class_weight_dict = dict(enumerate(class_weights))

model.fit(X_train, Y_train, epochs=10, batch_size=64, class_weight=class_weight_dict, validation_data=(X_test,Y_test), callbacks=[checkpoint])

Epoch 1/10
12/12 [==============================] - ETA: 0s - loss: 0.6710 - accuracy: 0.6411 
Epoch 1: val_loss improved from inf to 0.59976, saving model to hasoc_c3.h5
12/12 [==============================] - 168s 14s/step - loss: 0.6710 - accuracy: 0.6411 - val_loss: 0.5998 - val_accuracy: 0.7164
Epoch 2/10


C:\Users\8888\Anaconda3\envs\pythonProject11\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


12/12 [==============================] - ETA: 0s - loss: 0.6383 - accuracy: 0.6821 
Epoch 2: val_loss did not improve from 0.59976
12/12 [==============================] - 168s 14s/step - loss: 0.6383 - accuracy: 0.6821 - val_loss: 0.6054 - val_accuracy: 0.6567
Epoch 3/10
12/12 [==============================] - ETA: 0s - loss: 0.5828 - accuracy: 0.6702 
Epoch 3: val_loss improved from 0.59976 to 0.59414, saving model to hasoc_c3.h5
12/12 [==============================] - 165s 14s/step - loss: 0.5828 - accuracy: 0.6702 - val_loss: 0.5941 - val_accuracy: 0.6866
Epoch 4/10
12/12 [==============================] - ETA: 0s - loss: 0.5086 - accuracy: 0.7748 
Epoch 4: val_loss improved from 0.59414 to 0.50209, saving model to hasoc_c3.h5
12/12 [==============================] - 166s 14s/step - loss: 0.5086 - accuracy: 0.7748 - val_loss: 0.5021 - val_accuracy: 0.7985
Epoch 5/10
12/12 [==============================] - ETA: 0s - loss: 0.3924 - accuracy: 0.8556 
Epoch 5: val_loss did not impro

In [13]:
model.load_weights('hasoc_c3.h5')
model.evaluate(X_test,Y_test)

5/5 [==============================] - 4s 683ms/step - loss: 0.5021 - accuracy: 0.7985


[0.5020859241485596, 0.7985074520111084]

In [14]:
Y_pred = model.predict(test_X)

19/19 [==============================] - 14s 752ms/step


In [15]:
y_actual = []
for i in test_Y_true:
    if i == 'Vulgar':
        y_actual.append(1)
    else :
        y_actual.append(0)

pred_class = []
for i in Y_pred:
    pred_class.append(np.argmax(i))

In [16]:
print(classification_report(y_actual, pred_class))

              precision    recall  f1-score   support

           0       0.94      0.67      0.79       496
           1       0.35      0.81      0.49       108

    accuracy                           0.70       604
   macro avg       0.65      0.74      0.64       604
weighted avg       0.84      0.70      0.73       604



In [17]:
pred_actual = []
for i in pred_class:
    if i == 1:
        pred_actual.append('Vulgar')
    else :
        pred_actual.append('Non Vulgar')

In [18]:
test_data = test_data[["_id"]]
test_data["label"] = pred_actual
test_data.to_csv('dl_lstm_c3.csv',index=False)
test_data.head()

,_id,label
0,Gujarati_image_1225.jpg,Non Vulgar
1,Gujarati_image_1583.jpg,Vulgar
2,Gujarati_image_1502.jpg,Vulgar
3,Gujarati_image_1487.jpg,Vulgar
4,Gujarati_image_1497.jpg,Vulgar


# task_4

# Handling Pre-processed data

In [2]:
data = pd.read_csv('../preprocess_data.csv')
data.drop(['task_2','text'], axis=1, inplace=True)
data.head()

,_id,task_1,task_3,task_4,text_clean
0,Gujarati_image_1618.jpg,Sarcastic,Vulgar,Abusive,છોકર). ટીચર તમાર તાજમહેલ\r\n\r\nદેખ/ય છે.\r\n\...
1,Gujarati_image_31.jpg,Sarcastic,Vulgar,Abusive,છોકરો : ના.\r\n છોકરી : કેમ?\r\n \r\n છોકરી : ...
2,Gujarati_image_1144.jpg,Sarcastic,Vulgar,Abusive,"છોકરીઓ ગમે તેટલી\r\n ચાલક હોય,\r\n \r\n પણ છોક..."
3,Gujarati_image_1184.jpg,Sarcastic,Vulgar,Abusive,"દોસ્તી કરો,પ્રેમ કરો, વફા કરો...\r\n અને બહુ મ..."
4,Gujarati_image_1643.jpg,Sarcastic,Vulgar,Abusive,"છોકરીઓ ગમે તેટલી\r\nચાલક હોય,\r\n\r\nપણ છોકરા ..."


In [3]:
test_data = pd.read_csv('../preprocess_test_data.csv') 
test_data.drop(['task_2','text'], axis=1, inplace=True)
test_data = test_data.drop(['Unnamed: 0'],axis=1)
test_data.head()

,_id,task_1,task_3,task_4,text_clean
0,Gujarati_image_1225.jpg,Non-Sarcastic,Non Vulgar,Non-abusive,॥વિંદેશીગામડિયો\r\n અ |
1,Gujarati_image_1583.jpg,Sarcastic,Vulgar,Abusive,ટીચર : સૌથી વધારે દુખાવો ક્યારે\r\nથાય?\r\nછોક...
2,Gujarati_image_1502.jpg,Sarcastic,Vulgar,Abusive,પતિ: તુંમને જરાય પ્રેમ\r\nનથી કરતી...\r\n\r\nપ...
3,Gujarati_image_1487.jpg,Sarcastic,Vulgar,Abusive,આખા ગોમ ના લોડા\r\nભોસ મા ભરી ને બેઠી\r\nહોય અ...
4,Gujarati_image_1497.jpg,Non-Sarcastic,Vulgar,Abusive,મિનરલ વોટર સિવાય ક્યારેય\r\nબીજું\r\nપાણી નો પ...


In [4]:
sentences = data['text_clean'].astype(str)
tokenizer = Tokenizer(num_words = 1500,split=' ')
tokenizer.fit_on_texts(sentences)
sequence = tokenizer.texts_to_sequences(sentences)

In [5]:
test_sentences = test_data['text_clean'].astype(str)
test_sequence = tokenizer.texts_to_sequences(test_sentences)

In [6]:
max_seq_len = 2500

index_of_words = tokenizer.word_index
print("No of unique words : ",len(index_of_words))

X = pad_sequences(sequence , maxlen = max_seq_len )
Y = data['task_4']

print(X)

No of unique words :  7000
[[   0    0    0 ...  536  134    1]
 [   0    0    0 ...  246 1062  318]
 [   0    0    0 ... 1067 1068  174]
 ...
 [   0    0    0 ...    0    0    5]
 [   0    0    0 ...    0    0    5]
 [   0    0    0 ...    1   17  433]]


In [7]:
test_X = pad_sequences(test_sequence , maxlen = max_seq_len )
test_Y = test_data['task_4']

print(test_X)

[[  0   0   0 ...   0   0 648]
 [  0   0   0 ... 206 394   1]
 [  0   0   0 ...   2   1   1]
 ...
 [  0   0   0 ...   1   1 324]
 [  0   0   0 ...  40   2   1]
 [  0   0   0 ...   0   0   5]]


In [8]:
embed_dim = 256
vocabSize = len(index_of_words)
lstm_out = 64

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15, random_state = 0)
Y_true = Y_test
Y_train = pd.get_dummies(Y_train).values
Y_test = pd.get_dummies(Y_test).values

In [10]:
test_Y_true = test_Y
test_Y = pd.get_dummies(test_Y).values
print("test_Y:", test_Y)

test_Y: [[False  True]
 [ True False]
 [ True False]
 ...
 [False  True]
 [False  True]
 [False  True]]


# MODEL 1

In [11]:
model = Sequential()
model.add(Embedding(vocabSize, embed_dim,input_length = 2500))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2500, 256)         1792000   
                                                                 
 lstm (LSTM)                 (None, 64)                82176     
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 1874306 (7.15 MB)
Trainable params: 1874306 (7.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [12]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    "hasoc_a4.h5",
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False, 
    mode='auto')

In [13]:
model.fit(X_train, Y_train, batch_size = 32, epochs = 10, validation_data=(X_test,Y_test), callbacks=[checkpoint])

Epoch 1/10
24/24 [==============================] - ETA: 0s - loss: 0.5170 - accuracy: 0.7987
Epoch 1: val_loss improved from inf to 0.38842, saving model to hasoc_a4.h5
24/24 [==============================] - 99s 4s/step - loss: 0.5170 - accuracy: 0.7987 - val_loss: 0.3884 - val_accuracy: 0.8507
Epoch 2/10


C:\Users\8888\Anaconda3\envs\pythonProject11\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


24/24 [==============================] - ETA: 0s - loss: 0.4228 - accuracy: 0.8066
Epoch 2: val_loss improved from 0.38842 to 0.36340, saving model to hasoc_a4.h5
24/24 [==============================] - 97s 4s/step - loss: 0.4228 - accuracy: 0.8066 - val_loss: 0.3634 - val_accuracy: 0.8507
Epoch 3/10
24/24 [==============================] - ETA: 0s - loss: 0.3419 - accuracy: 0.8834
Epoch 3: val_loss improved from 0.36340 to 0.33369, saving model to hasoc_a4.h5
24/24 [==============================] - 97s 4s/step - loss: 0.3419 - accuracy: 0.8834 - val_loss: 0.3337 - val_accuracy: 0.8806
Epoch 4/10
24/24 [==============================] - ETA: 0s - loss: 0.2415 - accuracy: 0.9311
Epoch 4: val_loss did not improve from 0.33369
24/24 [==============================] - 98s 4s/step - loss: 0.2415 - accuracy: 0.9311 - val_loss: 0.3399 - val_accuracy: 0.8881
Epoch 5/10
24/24 [==============================] - ETA: 0s - loss: 0.1628 - accuracy: 0.9550
Epoch 5: val_loss did not improve from 0.

In [14]:
model.load_weights('hasoc_a4.h5')
model.evaluate(X_test,Y_test)

5/5 [==============================] - 2s 458ms/step - loss: 0.3337 - accuracy: 0.8806


[0.3336866497993469, 0.8805969953536987]

In [15]:
Y_pred = model.predict(test_X)

19/19 [==============================] - 10s 512ms/step


In [16]:
y_actual = []
for i in test_Y_true:
    if i == 'Non-abusive':
        y_actual.append(1)
    else :
        y_actual.append(0)

pred_class = []
for i in Y_pred:
    pred_class.append(np.argmax(i))

In [17]:
print(classification_report(y_actual , pred_class))

              precision    recall  f1-score   support

           0       0.81      0.19      0.31       205
           1       0.70      0.98      0.82       399

    accuracy                           0.71       604
   macro avg       0.76      0.58      0.56       604
weighted avg       0.74      0.71      0.64       604



In [18]:
print("Y_pred:", Y_pred)

Y_pred: [[0.00141402 0.99858606]
 [0.8391441  0.16085593]
 [0.30731696 0.69268304]
 ...
 [0.05582917 0.9441708 ]
 [0.00347404 0.99652594]
 [0.00159899 0.998401  ]]


In [19]:
print("pred_class:", pred_class)

pred_class: [1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [20]:
pred_actual = []
for i in pred_class:
    if i == 1:
        pred_actual.append('Non-abusive')
    else :
        pred_actual.append('Abusive')

In [21]:
test_data = test_data[["_id"]]
test_data["label"] = pred_actual
test_data.to_csv('dl_lstm_a4.csv',index=False)
test_data.head()

,_id,label
0,Gujarati_image_1225.jpg,Non-abusive
1,Gujarati_image_1583.jpg,Abusive
2,Gujarati_image_1502.jpg,Non-abusive
3,Gujarati_image_1487.jpg,Non-abusive
4,Gujarati_image_1497.jpg,Non-abusive


# MODEL 2

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential()

# Embedding层，增加防止过拟合的Dropout
model.add(Embedding(input_dim=vocabSize, output_dim=embed_dim, input_length=2500))
model.add(Dropout(0.3))

# LSTM层，增加recurrent_dropout 和 output_dropout
model.add(LSTM(units=lstm_out, dropout=0.3, recurrent_dropout=0.3, return_sequences=False))

# Batch Normalization增强泛化
model.add(BatchNormalization())

# 全连接层，Softmax输出2分类，建议用categorical_crossentropy
model.add(Dense(2, activation='softmax'))

# 编译
optimizer = Adam(learning_rate=0.001)  # 学习率也可调整
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2500, 256)         1792000   
                                                                 
 dropout (Dropout)           (None, 2500, 256)         0         
                                                                 
 lstm (LSTM)                 (None, 64)                82176     
                                                                 
 batch_normalization (Batch  (None, 64)                256       
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 1874562 (7.15 MB)
Trainable params: 1874434 (7.15 MB)
Non-trainable params: 128 (512.00 Byte)
______________

In [12]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    "hasoc_b4.h5",
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False, 
    mode='auto')

In [13]:
model.fit(X_train, Y_train, batch_size = 32, epochs = 10, validation_data=(X_test,Y_test), callbacks=[checkpoint])

Epoch 1/10
24/24 [==============================] - ETA: 0s - loss: 0.7174 - accuracy: 0.5748
Epoch 1: val_loss improved from inf to 0.57835, saving model to hasoc_b4.h5
24/24 [==============================] - 99s 4s/step - loss: 0.7174 - accuracy: 0.5748 - val_loss: 0.5783 - val_accuracy: 0.8507
Epoch 2/10


C:\Users\8888\Anaconda3\envs\pythonProject11\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


24/24 [==============================] - ETA: 0s - loss: 0.4703 - accuracy: 0.8106
Epoch 2: val_loss improved from 0.57835 to 0.50496, saving model to hasoc_b4.h5
24/24 [==============================] - 97s 4s/step - loss: 0.4703 - accuracy: 0.8106 - val_loss: 0.5050 - val_accuracy: 0.8507
Epoch 3/10
24/24 [==============================] - ETA: 0s - loss: 0.3282 - accuracy: 0.9166
Epoch 3: val_loss improved from 0.50496 to 0.44701, saving model to hasoc_b4.h5
24/24 [==============================] - 97s 4s/step - loss: 0.3282 - accuracy: 0.9166 - val_loss: 0.4470 - val_accuracy: 0.8507
Epoch 4/10
24/24 [==============================] - ETA: 0s - loss: 0.2050 - accuracy: 0.9404
Epoch 4: val_loss improved from 0.44701 to 0.41452, saving model to hasoc_b4.h5
24/24 [==============================] - 97s 4s/step - loss: 0.2050 - accuracy: 0.9404 - val_loss: 0.4145 - val_accuracy: 0.8507
Epoch 5/10
24/24 [==============================] - ETA: 0s - loss: 0.1786 - accuracy: 0.9430
Epoch 5:

In [14]:
model.load_weights('hasoc_b4.h5')
model.evaluate(X_test,Y_test)

5/5 [==============================] - 2s 424ms/step - loss: 0.3597 - accuracy: 0.8582


[0.35971522331237793, 0.858208954334259]

In [15]:
Y_pred = model.predict(test_X)

19/19 [==============================] - 9s 473ms/step


In [16]:
y_actual = []
for i in test_Y_true:
    if i == 'Non-abusive':
        y_actual.append(1)
    else :
        y_actual.append(0)

pred_class = []
for i in Y_pred:
    pred_class.append(np.argmax(i))

In [17]:
print(classification_report(y_actual , pred_class))

              precision    recall  f1-score   support

           0       0.95      0.10      0.19       205
           1       0.68      1.00      0.81       399

    accuracy                           0.69       604
   macro avg       0.82      0.55      0.50       604
weighted avg       0.78      0.69      0.60       604



In [18]:
pred_actual = []
for i in pred_class:
    if i == 1:
        pred_actual.append('Non-abusive')
    else :
        pred_actual.append('Abusive')

In [19]:
test_data = test_data[["_id"]]
test_data["label"] = pred_actual
test_data.to_csv('dl_lstm_b4.csv',index=False)
test_data.head()

,_id,label
0,Gujarati_image_1225.jpg,Non-abusive
1,Gujarati_image_1583.jpg,Abusive
2,Gujarati_image_1502.jpg,Non-abusive
3,Gujarati_image_1487.jpg,Non-abusive
4,Gujarati_image_1497.jpg,Non-abusive


# MODEL 3

In [11]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    "hasoc_c4.h5",
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False, 
    mode='auto')

In [12]:
model = Sequential()
model.add(Embedding(vocabSize, embed_dim, input_length=2500))
model.add(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

from sklearn.utils.class_weight import compute_class_weight
# 假设Y_train已独热编码
class_weights = compute_class_weight(class_weight='balanced', classes=np.array([0, 1]), y=np.argmax(Y_train, axis=1))
class_weight_dict = dict(enumerate(class_weights))

model.fit(X_train, Y_train, epochs=10, batch_size=64, class_weight=class_weight_dict, validation_data=(X_test,Y_test), callbacks=[checkpoint])

Epoch 1/10
12/12 [==============================] - ETA: 0s - loss: 0.6711 - accuracy: 0.6755 
Epoch 1: val_loss improved from inf to 0.59755, saving model to hasoc_c4.h5
12/12 [==============================] - 171s 14s/step - loss: 0.6711 - accuracy: 0.6755 - val_loss: 0.5975 - val_accuracy: 0.6269
Epoch 2/10


C:\Users\8888\Anaconda3\envs\pythonProject11\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


12/12 [==============================] - ETA: 0s - loss: 0.6163 - accuracy: 0.7007 
Epoch 2: val_loss did not improve from 0.59755
12/12 [==============================] - 163s 13s/step - loss: 0.6163 - accuracy: 0.7007 - val_loss: 0.6313 - val_accuracy: 0.5896
Epoch 3/10
12/12 [==============================] - ETA: 0s - loss: 0.5556 - accuracy: 0.7510 
Epoch 3: val_loss improved from 0.59755 to 0.53907, saving model to hasoc_c4.h5
12/12 [==============================] - 164s 14s/step - loss: 0.5556 - accuracy: 0.7510 - val_loss: 0.5391 - val_accuracy: 0.7313
Epoch 4/10
12/12 [==============================] - ETA: 0s - loss: 0.4280 - accuracy: 0.8464 
Epoch 4: val_loss improved from 0.53907 to 0.52551, saving model to hasoc_c4.h5
12/12 [==============================] - 164s 14s/step - loss: 0.4280 - accuracy: 0.8464 - val_loss: 0.5255 - val_accuracy: 0.8134
Epoch 5/10
12/12 [==============================] - ETA: 0s - loss: 0.2968 - accuracy: 0.9430 
Epoch 5: val_loss did not impro

In [13]:
model.load_weights('hasoc_c4.h5')
model.evaluate(X_test,Y_test)

5/5 [==============================] - 4s 695ms/step - loss: 0.4379 - accuracy: 0.8582


[0.437895804643631, 0.858208954334259]

In [14]:
Y_pred = model.predict(test_X)

19/19 [==============================] - 15s 785ms/step


In [15]:
y_actual = []
for i in test_Y_true:
    if i == 'Non-abusive':
        y_actual.append(1)
    else :
        y_actual.append(0)

pred_class = []
for i in Y_pred:
    pred_class.append(np.argmax(i))

In [16]:
print(classification_report(y_actual , pred_class))

              precision    recall  f1-score   support

           0       0.68      0.35      0.46       205
           1       0.73      0.91      0.81       399

    accuracy                           0.72       604
   macro avg       0.71      0.63      0.64       604
weighted avg       0.71      0.72      0.69       604



In [17]:
pred_actual = []
for i in pred_class:
    if i == 1:
        pred_actual.append('Non-abusive')
    else :
        pred_actual.append('Abusive')

In [18]:
test_data = test_data[["_id"]]
test_data["label"] = pred_actual
test_data.to_csv('dl_lstm_c4.csv',index=False)
test_data.head()

,_id,label
0,Gujarati_image_1225.jpg,Non-abusive
1,Gujarati_image_1583.jpg,Abusive
2,Gujarati_image_1502.jpg,Abusive
3,Gujarati_image_1487.jpg,Abusive
4,Gujarati_image_1497.jpg,Abusive
